# Demodulacija FSK kroz FFT -- v1, trivial, no sync yet

### biblioteke

In [ ]:
import numpy as np
import fastgoertzel as G
from scipy.io import wavfile
from ipyfilechooser import FileChooser
import matplotlib.pyplot as plt
import os
from pathlib import Path
from os.path import exists
# import librosa
import scipy.fft as fft
import scipy
from scipy.signal import remez,freqz,firwin
import decimal
from scipy.signal import find_peaks
# from scipy.signal import findfre2
import binascii
from scipy import signal
%matplotlib widget

### bitappender, bits to bytes

In [ ]:
class BitAppender:
    def __init__(self):
        self.buffer = bytearray()
        self.current_byte = 0
        self.bit_count = 0

    def append_bit(self, bit):
        if bit not in (0, 1):
            raise ValueError("Bit must be 0 or 1")

        # Add the bit to the current byte
        self.current_byte = (self.current_byte << 1) | bit
        self.bit_count += 1

        # If we have accumulated 8 bits, store the byte and reset
        if self.bit_count == 8:
            self.buffer.append(self.current_byte)
            self.current_byte = 0
            self.bit_count = 0

    def get_bytes(self):
        # If there are leftover bits, pad with zeros and add the final byte
        if self.bit_count > 0:
            self.current_byte <<= (8 - self.bit_count)
            self.buffer.append(self.current_byte)
        return bytes(self.buffer)

# Example usage:
# bit_appender = BitAppender()
# bits = [1, 0, 1, 1, 0, 0, 1, 0,  # 0b10110010 (178 in decimal)
#         1, 1, 1, 1, 0, 0, 1, 1]  # 0b11110011 (243 in decimal)

# for bit in bits:
#     bit_appender.append_bit(bit)

# result = bit_appender.get_bytes()
# print(result)  # Output: b'\xb2\xf3'

### selektor i file picker

In [ ]:
def biraj_folder() -> FileChooser:
    # biranje foldera
    fc = FileChooser()
    fc.show_only_dirs = True

    # brza pretraga da nadje default folder, sto da ne
    for dirname,_,_ in os.walk(fc.default_path):
        if Path(dirname).name == "wav":
            fc.default_path = dirname
            fc._select_default = True  ## da ne moramo da selektujemo svaki put
            fc.reset()
            break
    # ovo valjda odradi posao
    display(fc)
    return fc

def ucitaj_fajl(fc:FileChooser):
    global samplerate, sig
    # samplerate = sr
    # sig = sig
    # zapravo ucitavanje fajla ## mozda bi valjalo da resimo da se radi kroz neki update automatski kad selektujes
    selected = os.fsencode(fc.selected)
    # filename = os.fsdecode(file)

    if os.path.isfile(selected):
        try:
            # sig, samplerate = librosa.load(fc.selected + filename) 
            samplerate, sig = wavfile.read(fc.selected) 
            return(samplerate, sig)
            

        except:
            print("greska sa citanjem fajla:" + fc.selected)
    

def biraj_fajl():
    # biranje fajla
    fc = FileChooser()
    fc.register_callback(ucitaj_fajl)   
    display(fc)
    # return fc
biraj_fajl()

### spektogram za proveru

In [ ]:
plt.clf()
plt.specgram(sig,Fs=samplerate )
plt.show()

### trivial fft demod, nosync, fixed experimental tresh

In [ ]:
dt = 0.1
f0 = 9500
f1 = 10500
peak_tresh = 10000000

bit_appender = BitAppender()
small_segment = int(samplerate * dt  * 0.01)
small_segment = 50 # mali delic tako da ne uhvatimo prelaze frekvencija i olaksamo potrebe sinhronizacije
small_segment = 100

N = len(sig)

T = 1.0 / samplerate
# x = np.linspace(0.0, N*T, N)

yf = scipy.fftpack.fft(sig)
# xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

bl = []

for i in np.arange(0,len(sig)/samplerate,dt):

    y1 = sig[int(i*samplerate + small_segment) : int((i+dt)*samplerate) - small_segment]/1
    y1fft = scipy.fft.rfft(y1)

    n = int(dt * samplerate)- 2 * small_segment

    freqs = fft.rfftfreq(n,1./samplerate)
    peaks,props = find_peaks(y1fft,height = peak_tresh)

    ### modifikovati, trenutno preskace veliki peak kod dc-a ovako, napraviti bolje resenje jer ovo nece raditi za nize fs i brzine odbiranja
    if f0-300 < freqs[y1fft[15:].argmax()] < f0 + 300: bit_appender.append_bit(0);bl.append(0)
    if f1-300 < freqs[y1fft[15:].argmax()] < f1 +300: bit_appender.append_bit(1);bl.append(1)
    

result = bit_appender.get_bytes()
print(bl)
print(result)  # Output: b'\xb2\xf3'